In [1]:
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph


graph = Neo4jGraph(
    url="bolt://44.204.234.176",
    username="neo4j", 
    password="screens-tackle-lighter"
)

In [2]:
from langchain.llms import VertexAI


llm = VertexAI(project="gci-generative-ai-b", location="us-central1",top_p=0.95,top_k=40,max_output_tokens=128,temperature=0.0)

chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True,
)

In [3]:
chain.run("""
Which fortnite streamer has the most followers?
""")



> Entering new  chain...
Generated Cypher:
MATCH (s:Stream)-[:PLAYS]->(g:Game {name: "Fortnite"}) WITH s ORDER BY s.followers DESC LIMIT 1 RETURN s.name
Full Context:
[{'s.name': 'thegrefg'}]

> Finished chain.


'The most followed Fortnite streamer is TheGrefg.'

In [4]:
chain.run("""
Which italian streamer has the most followers?
""")



> Entering new  chain...
Generated Cypher:
MATCH (s:Stream)-[:HAS_LANGUAGE]->(l:Language {name: "Italian"})-[:HAS_STREAM]->(u:User) RETURN u.name, s.followers ORDER BY s.followers DESC LIMIT 1
Full Context:
[]

> Finished chain.


"I don't know the answer."

In [5]:
# https://github.com/hwchase17/langchain/blob/master/langchain/chains/graph_qa/prompts.py
from langchain.prompts.prompt import PromptTemplate


CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Cypher examples:
# How many streamers are from Norway?
MATCH (s:Stream)-[:HAS_LANGUAGE]->(:Language {{name: 'no'}})
RETURN count(s) AS streamers

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

The question is:
{question}"""
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

In [6]:
chain_language_example = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT
)

chain_language_example.run("""
Which italian streamer has the most followers?
""")




> Entering new  chain...
Generated Cypher:
MATCH (s:Stream)-[:HAS_LANGUAGE]->(:Language {name: 'it'})
RETURN s.name, s.followers ORDER BY s.followers DESC LIMIT 1
Full Context:
[{'s.name': 'pow3rtv', 's.followers': 1530428}]

> Finished chain.


'The Italian streamer with the most followers is pow3rtv.'

In [7]:
chain_language_example.run("""
Which streamers should I also watch if I like pokimane?
""")



> Entering new  chain...
Generated Cypher:
MATCH (s:Stream {name: 'pokimane'})-[:CHATTER]->(u:User)
MATCH (u)-[:CHATTER]->(s2:Stream)
RETURN s2.name AS recommended_streamers
Full Context:
[]

> Finished chain.


"I don't know the answer."

In [8]:
# Project in-memory graph
graph.query("""
CALL gds.graph.project('shared-audience',
  ['User', 'Stream'],
  {CHATTER: {orientation:'REVERSE'}})
""")

# Run node similarity algorithm
graph.query("""
CALL gds.nodeSimilarity.mutate('shared-audience',
 {similarityMetric: 'Jaccard',similarityCutoff:0.05, topK:10, sudo:true,
     mutateProperty:'score', mutateRelationshipType:'SHARED_AUDIENCE'})
""")

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.project`: Caused by: java.lang.IllegalArgumentException: A graph with name 'shared-audience' already exists.}

In [9]:
graph.query("""
MATCH (s:Stream)
WHERE s.name = "kimdoe"
WITH collect(s) AS sourceNodes
CALL gds.pageRank.stream("shared-audience", 
  {sourceNodes:sourceNodes, relationshipTypes:['SHARED_AUDIENCE'], 
    nodeLabels:['Stream']})
YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS node, score
WHERE NOT node in sourceNodes
RETURN node.name AS streamer, score
ORDER BY score DESC LIMIT 3
""")

[{'streamer': 'jungtaejune', 'score': 0.13697276805472167},
 {'streamer': 'tranth', 'score': 0.13697276805472167},
 {'streamer': 'hanryang1125', 'score': 0.10511818935406857}]

In [10]:
# https://github.com/hwchase17/langchain/blob/master/langchain/chains/graph_qa/prompts.py

CYPHER_RECOMMENDATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Cypher examples:
# How many streamers are from Norway?
MATCH (s:Stream)-[:HAS_LANGUAGE]->(:Language {{name: 'no'}})
RETURN count(s) AS streamers
# Which streamers do you recommend if I like kimdoe?
MATCH (s:Stream)
WHERE s.name = "kimdoe"
WITH collect(s) AS sourceNodes
CALL gds.pageRank.stream("shared-audience", 
  {{sourceNodes:sourceNodes, relationshipTypes:['SHARED_AUDIENCE'], 
    nodeLabels:['Stream']}})
YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS node, score
WHERE NOT node in sourceNodes
RETURN node.name AS streamer, score
ORDER BY score DESC LIMIT 3

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

The question is:
{question}"""
CYPHER_RECOMMENDATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_RECOMMENDATION_TEMPLATE
)

In [11]:
chain_recommendation_example = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True,
    cypher_prompt=CYPHER_RECOMMENDATION_PROMPT, 
)

chain_recommendation_example.run("""
Which streamers do you recommend if I like pokimane?
""")



> Entering new  chain...
Generated Cypher:
MATCH (s:Stream)
WHERE s.name = "pokimane"
WITH collect(s) AS sourceNodes
CALL gds.pageRank.stream("shared-audience", 
  {sourceNodes:sourceNodes, relationshipTypes:['SHARED_AUDIENCE'], 
    nodeLabels:['Stream']})
YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS node, score
WHERE NOT node in sourceNodes
RETURN node.name AS streamer, score
ORDER BY score DESC LIMIT 3
Full Context:
[{'streamer': 'xchocobars', 'score': 0.2343657053097286}, {'streamer': 'ariasaki', 'score': 0.06485239618458194}, {'streamer': 'natsumiii', 'score': 0.05969369486512491}]

> Finished chain.


'I recommend xchocobars, ariasaki, and natsumiii.'

In [12]:
chain_recommendation_example.run("""
Which streamers do you recommend to watch if I like Chess games?
""")



> Entering new  chain...
Generated Cypher:
MATCH (s:Stream)-[:PLAYS]->(g:Game {name: "Chess"})
RETURN s.name AS streamer
Full Context:
[{'streamer': 'thisisnotgeorgenotfound'}, {'streamer': 'gmharikrishna'}, {'streamer': 'michelleputtini'}, {'streamer': 'alirezafirouzja'}, {'streamer': 'chesscoachandras'}, {'streamer': 'attilaturzo'}, {'streamer': 'gmneiksans'}, {'streamer': 'joebruin'}, {'streamer': 'd2thafool'}, {'streamer': 'imsatranc'}]

> Finished chain.


'I recommend watching thisisnotgeorgenotfound, gmharikrishna, michelleputtini, alirezafirouzja, chesscoachandras, attilaturzo, gmneiksans, joebruin, d2thafool, and imsatranc.'